In [2]:
from sklearn.datasets import fetch_20newsgroups
from matplotlib import pyplot as plt
from collections import Counter
newsgroups_train = fetch_20newsgroups(subset='train')
newsgroups_test = fetch_20newsgroups(subset='test')
from keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
import numpy as np
import gensim
import os, re, csv, math, codecs
import numpy as np
import tensorflow as tf
from sklearn.preprocessing import LabelEncoder
from tensorflow.keras.layers import Input, Embedding, Conv1D, GlobalMaxPooling1D, Bidirectional, LSTM, Attention, Dense
from tensorflow.keras.models import Model
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.callbacks import EarlyStopping
from keras.layers import Embedding, Conv1D, MaxPooling1D, GlobalMaxPooling1D, Dropout, Dense, Lambda
from keras.models import Sequential,Model
from keras import optimizers
import keras.backend as K

In [3]:
token=Tokenizer(num_words=30000, filters='!"#$%&()*+,-./:;<=>?@[\\]^_`{|}~\t\n', lower=True, split=' ', char_level=False, oov_token="UNK", document_count=0)

In [4]:
token.fit_on_texts(newsgroups_train.data)

In [5]:
train_sequences=token.texts_to_sequences(newsgroups_train.data)
test_sequences=token.texts_to_sequences(newsgroups_test.data)


In [6]:
max_len=1000
train_sequences=pad_sequences(train_sequences,maxlen=max_len)
test_sequences=pad_sequences(test_sequences,maxlen=max_len)

In [7]:
reverse_dictionary = token.index_word
dictionary = dict([(value, key) for (key, value) in reverse_dictionary.items()]) 

In [8]:
embed_dim=300
num_words=len(dictionary)+1

In [35]:
nb_words=num_words
num_filters=64
model = Sequential()
model.add(Embedding(nb_words, embed_dim, input_length=max_len, trainable=True)) #trainable=False
model.add(Lambda(lambda x: K.sum(x, axis=1)))
model.add(Dense(2000, activation='relu'))
model.add(Dense(2000, activation='relu'))
model.add(Dense(2000, activation='relu'))
model.add(Dense(20, activation='softmax'))  #multi-label (k-hot encoding)

adam = optimizers.Adam(lr=0.001, beta_1=0.9, beta_2=0.999, epsilon=1e-08, decay=0.0)
model.compile(loss='sparse_categorical_crossentropy', optimizer=adam, metrics=['accuracy'])
model.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_5 (Embedding)     (None, 1000, 300)         40243200  
                                                                 
 lambda_1 (Lambda)           (None, 300)               0         
                                                                 
 dense_8 (Dense)             (None, 2000)              602000    
                                                                 
 dense_9 (Dense)             (None, 2000)              4002000   
                                                                 
 dense_10 (Dense)            (None, 2000)              4002000   
                                                                 
 dense_11 (Dense)            (None, 20)                40020     
                                                                 
Total params: 48,889,220
Trainable params: 48,889,220


c:\Python311\Lib\site-packages\keras\optimizers\legacy\adam.py:117: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super().__init__(name, **kwargs)


In [9]:
early_stopping = EarlyStopping(patience=3, restore_best_weights=True)

In [40]:
model.fit(train_sequences, newsgroups_train.target,batch_size=28,epochs=30,validation_split=0.2,callbacks=[early_stopping])

Epoch 1/30
324/324 [==============================] - 270s 834ms/step - loss: 1.2623 - accuracy: 0.5760 - val_loss: 0.8811 - val_accuracy: 0.7446
Epoch 2/30
324/324 [==============================] - 253s 781ms/step - loss: 0.4847 - accuracy: 0.8466 - val_loss: 0.8716 - val_accuracy: 0.7707
Epoch 3/30
 16/324 [>.............................] - ETA: 4:25 - loss: 0.2453 - accuracy: 0.9330

KeyboardInterrupt: 

In [ ]:
import tensorflow as tf
model.save('MLP+embeddings.h5') 
K.clear_session()

In [10]:
K.clear_session

<function keras.backend.clear_session()>

In [11]:
from keras.layers import Embedding, Conv1D, MaxPooling1D, GlobalMaxPooling1D, Dropout, Dense, Input, Concatenate
from keras.models import Model
from keras import optimizers
nb_words=num_words
num_filters=64
input_layer = Input(shape=(max_len,))
embedding_layer=Embedding(nb_words, embed_dim, input_length=max_len, trainable=True)(input_layer)
conv4=Conv1D(num_filters, 4, activation='relu', padding='same')(embedding_layer)
conv3=Conv1D(num_filters, 3, activation='relu', padding='same')(embedding_layer)
conv2=Conv1D(num_filters, 2, activation='relu', padding='same')(embedding_layer)

attention4 = Attention()([conv4, conv4])
attention3 = Attention()([conv3, conv3])
attention2 = Attention()([conv2, conv2])

pool4 = GlobalMaxPooling1D()(attention4)
pool3 = GlobalMaxPooling1D()(attention3)
pool2 = GlobalMaxPooling1D()(attention2)

added = Concatenate()([pool4, pool3, pool2])
dense1=Dense(32, activation='relu')(added)
dense2=Dense(20, activation='softmax')(dense1)
model3=Model(input_layer , dense2)
adam = optimizers.Adam(lr=0.001, beta_1=0.9, beta_2=0.999, epsilon=1e-08, decay=0.0)
model3.compile(loss='sparse_categorical_crossentropy', optimizer=adam, metrics=['accuracy'])

c:\Python311\Lib\site-packages\keras\optimizers\legacy\adam.py:117: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super().__init__(name, **kwargs)


In [12]:
model3.fit(train_sequences, newsgroups_train.target,batch_size=128,epochs=20,validation_split=0.2,callbacks=[early_stopping])

Epoch 1/20
71/71 [==============================] - 868s 12s/step - loss: 2.8506 - accuracy: 0.1041 - val_loss: 2.3362 - val_accuracy: 0.3080
Epoch 2/20
71/71 [==============================] - 936s 13s/step - loss: 1.3959 - accuracy: 0.5975 - val_loss: 0.8620 - val_accuracy: 0.7715
Epoch 3/20
71/71 [==============================] - 986s 14s/step - loss: 0.4780 - accuracy: 0.8734 - val_loss: 0.5740 - val_accuracy: 0.8445
Epoch 4/20
71/71 [==============================] - 917s 13s/step - loss: 0.1748 - accuracy: 0.9612 - val_loss: 0.5357 - val_accuracy: 0.8577
Epoch 5/20
71/71 [==============================] - 963s 14s/step - loss: 0.0639 - accuracy: 0.9886 - val_loss: 0.5042 - val_accuracy: 0.8683
Epoch 6/20
35/71 [=============>................] - ETA: 7:38 - loss: 0.0251 - accuracy: 0.9980


KeyboardInterrupt



In [ ]:
import tensorflow as tf
model3.save('TCNN+Attention.h5') 
K.clear_session()